# Title
The title of the notebook should be coherent with file name. Namely, file name should be:    
*author's initials_progressive number_title.ipynb*    
For example:    
*EF_01_Data Exploration.ipynb*

## Purpose
State the purpose of the notebook.

## Methodology
Quickly describe assumptions and processing steps.

## WIP - improvements
Use this section only if the notebook is not final.

Notable TODOs:
- todo 1;
- todo 2;
- todo 3.

## Results
Describe and comment the most important results.

## Suggested next steps
State suggested next steps, based on results obtained in this notebook.

# Setup

## Library import
We import all the required Python libraries

In [3]:
import os
from typing import List

# Data manipulation
import pandas as pd
import numpy as np

# Visualizations
import matplotlib as plt
from pandas_profiling import ProfileReport
import plotly
import plotly.graph_objs as go
import plotly.offline as ply
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split

os.chdir('../')
from src.utils.data_describe import breve_descricao, serie_nulos, cardinalidade
os.chdir('./notebooks/')

# Options for pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

plotly.offline.init_notebook_mode(connected=True)

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.

In [4]:
RAW_FOLDER = '../data/raw/'
INTERIM_FOLDER = '../data/interim/'
REPORTS_FOLDER = '../reports/'
RANDOM_STATE = 42


# Data import
We retrieve all the required data for the analysis.

In [5]:
df = pd.read_parquet(INTERIM_FOLDER + 'df_train_interim_01_baseline.pqt')
df_evaluation = df.copy() 
df_evaluation.shape

(1460, 19)

In [6]:
df_evaluation.columns

Index(['LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'PoolArea', 'YrSold',
       'YearBuilt_YrSold', 'YearRemodAdd_YrSold', 'GarageYrBlt_YrSold',
       'SalePrice'],
      dtype='object')

## Splitting data in training and validation datasets

In [7]:
X = df_evaluation.drop(columns=['SalePrice'])
y = df_evaluation[['SalePrice']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

# Modeling


In [12]:
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluating

In [15]:
print(f"RMSE: {mse(y_test, y_pred)**.5:.2f}")

RMSE: 40198.72
